# Load and save data

Up until now, we always explicitly specified file names. If we want something more flexible (e.g., when working with multiple subjects, there is a nice interface for that:
* [SelectFiles](http://nipy.org/nipype/users/select_files.html)


---
Up until now, we only lookt at data in the working directory. Usually, we are not interested in intermediate results. We can create an output directory that has only the data we want to store permatently:
* [Data Sink](http://nipy.org/nipype/interfaces/generated/nipype.interfaces.io.html#datasink)

In [ ]:
import os
try:
    os.chdir(notebook_path)
except:
    notebook_path = os.path.abspath('.')

output_path = os.path.abspath('outputs')
if not os.path.exists(output_path):
    os.mkdir(output_path)
    
wd_path = os.path.join(output_path, '04_workflow')
if not os.path.exists(wd_path):
    os.mkdir(wd_path)
os.chdir(wd_path)
print(wd_path)

# Workflow

Let's take the previous workflow 

In [ ]:
from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces.nipy.preprocess import Trim, FmriRealign4d

workflow_path = os.path.join(wd_path, 'wf_3')
wf = Workflow(name='load_save_wf')
wf.base_dir = workflow_path
wf.config['execution']['crashdump_dir'] = os.path.join(workflow_path, 'crash')
wf.config['execution']['remove_unnecessary_outputs'] = False

In [ ]:
trim = Node(Trim(), 'trim')
trim.inputs.end_index = 10

realign = Node(FmriRealign4d(), 'realign')
realign.inputs.tr = 3.
realign.inputs.time_interp = True
realign.inputs.slice_order = range(0,35)

wf.connect(trim, 'out_file', realign, 'in_file')

Note that we only connected the two nodes, we did not define input data

# SelectFiles

In [ ]:
from nipype.interfaces.io import DataSink, SelectFiles

In [ ]:
data_path = os.path.join(notebook_path, 'data')

file_template = {'funct': 'ds107/sub001/BOLD/task001_run001/bold.nii.gz'}

selectfiles = Node(SelectFiles(file_template, base_directory=data_path),
                   name="selectfiles")

Now we connect selectfiles to moco:

In [ ]:
wf.connect(selectfiles, 'funct', trim, 'in_file')

If we had more than one file we could define the template like this:

In [ ]:
file_template = {'funct': 'in_data/s01/rs_10TRs.nii.gz',
                'anat':'data/ds107/sub001/anatomy/highres001.nii.gz'}

# DataSink

In [ ]:
ds_dir = os.path.join(wd_path, 'output_data')

ds = Node(DataSink(), name='ds')
ds.inputs.base_directory = ds_dir

compare the style of the base_directory definintion of selectfiles. They work interchangeably

Now we connect the output of smoothing with datasink

In [ ]:
wf.connect(realign, 'out_file', ds, 'realigned_data')

and run the workflow

In [ ]:
wf.run()